In [ ]:
import subprocess
import xml.etree.ElementTree as ET

In [ ]:
# Generate original files
with open('data/TransCoder_tokenized_test_set_functions/test.java.shuf.valid.tok', 'r') as f:
    data = f.readlines()

for item in data:
    k_and_v = item.split(' | ', 1)
    k, v = k_and_v[0], k_and_v[1]
    with open('java-python/test/original/' + k + '.java', 'w') as f:
        f.write(v)

In [ ]:
# Generate input XML files
import IPython
for entry in os.scandir('experiment/java-python/test/original'):
    print(entry.name)
    # Read Java file and srcML it to XML
    with open('experiment/java-python/test/xml_input/' + entry.name.split('.')[0] + '.xml', 'w+') as f:
        subprocess.run(['srcml', entry.path], stdout=f)
    IPython.display.clear_output()

In [ ]:
# Transformation
for entry in os.scandir('experiment/java-python/test/xml_input'):
    # Parse into AST
    try:
        tree = ET.parse(entry.path)
    except Exception as e:
        print(entry.name)
        print(e)
        continue
    root = tree.getroot()
    ns = {'src': 'http://www.srcML.org/srcML/src'}

    num_do, num_for, num_if, num_switch, num_while = 0, 0, 0, 0, 0

    for elem in root.iter():
        for item in elem.findall('src:do', ns):
            elem.remove(item)
            tree.write('experiment/java-python/test/xml_output/do__' + str(num_do) + '__' + entry.name)
            num_do += 1

        for item in elem.findall('src:for', ns):
            elem.remove(item)
            tree.write('experiment/java-python/test/xml_output/for__' + str(num_for) + '__' + entry.name)
            num_for += 1
        
        for item in elem.findall('src:if_stmt', ns):
            elem.remove(item)
            tree.write('experiment/java-python/test/xml_output/if__' + str(num_if) + '__' + entry.name)
            num_if += 1

        for item in elem.findall('src:switch', ns):
            elem.remove(item)
            tree.write('experiment/java-python/test/xml_output/switch__' + str(num_switch) + '__' + entry.name)
            num_switch += 1
    
        for item in elem.findall('src:while', ns):
            elem.remove(item)
            tree.write('experiment/java-python/test/xml_output/while__' + str(num_while) + '__' + entry.name)
            num_while += 1

In [ ]:
# Output XML files to Java output
for entry in os.scandir('experiment/java-python/test/xml_output'):
    with open('experiment/java-python/test/mutants/' + entry.name.split('.')[0] + '.java', 'w+') as f:
        subprocess.run(['srcml', entry.path], stdout=f)

In [ ]:
# Translate

In [ ]:
import IPython
import pandas as pd
import subprocess

d = {'key': [], 'Computational accuracy': []}
# blocklist = ['for__0__MOVE_ZEROES_END_ARRAY.py', 'if__0__CHECK_IF_A_NUMBER_IS_POWER_OF_ANOTHER_NUMBER.py', 'if__1__MODULUS_TWO_FLOAT_DOUBLE_NUMBERS.py', 'if__0__ALTERNATIVE_SORTING.py', 'if__2__MODULUS_TWO_FLOAT_DOUBLE_NUMBERS.py', 'while__0__COUNT_FACTORIAL_NUMBERS_IN_A_GIVEN_RANGE.py', 'while__1__K_TH_ELEMENT_TWO_SORTED_ARRAYS.py']
blocklist = ['SUM_K_TH_GROUP_ODD_POSITIVE_NUMBERS.py', 'ALTERNATIVE_SORTING.py', 'MINIMUM_LENGTH_SUBARRAY_SUM_GREATER_GIVEN_VALUE.py', 'MOVE_ZEROES_END_ARRAY.py', 'NUMBER_IS_DIVISIBLE_BY_29_OR_NOT.py', 'PRINT_WORDS_STRING_REVERSE_ORDER.py', 'NUMBER_TRIANGLES_N_MOVES_1.py', 'MINIMUM_LENGTH_SUBARRAY_SUM_GREATER_GIVEN_VALUE_1.py']

for entry in os.scandir('experiment/java-python/test/tests'):
    if entry.name not in blocklist:
        print(entry.name)
        out = subprocess.run(['python3', entry.path], capture_output=True).stdout
        out = out.decode()
        if out:
            if out[0] == '#':
                unkilled = float(out.split(', ')[0].split(' ')[1])
                total = float(out.split(', ')[1].split('\n')[0])
                # mutation_score = (total - unkilled) / total
                mutation_score = unkilled / total
                
                d['key'].append(entry.name.split('.')[0])
                d['Computational accuracy'].append(mutation_score)

        IPython.display.clear_output()

df = pd.DataFrame(data=d)
df

In [ ]:
df3 = pd.concat([df, df2])
df3.describe()

In [ ]:
df3.plot.hist()

In [ ]:
df3.to_csv('experiment/java-python/result/original.csv')

In [ ]:
# class wrapper for j2py translation
for entry in os.scandir('experiment/java-python/test/mutants'):
    with open(entry.path) as f:
        with open('experiment/java-python/j2pyraw/' + entry.name, 'w') as fi:
            fi.write('class X { ' + f.readline() + ' }')

In [ ]:

import subprocess
import IPython
for entry in os.scandir('experiment/java-python/j2pyraw'):
    print(entry.name)
    subprocess.run(['j2py', entry.path, 'experiment/java-python/j2pyrawpy/' + entry.name.split('.')[0] + '.py'])
    IPython.display.clear_output()

In [ ]:
# Insert j2py tests
import subprocess
import IPython
import re
for entry in os.scandir('experiment/java-python/j2pyrawpy'):
    desc = entry.name.split('__')[2]
    print(entry.name)

    with open(entry.path) as f:
        lines = f.readlines()
        ind = int()
        for item in lines:
            if item.startswith('        \"\"\" generated source for method '):
                ind = lines.index(item) - 1
                break
        data = [item.replace('    ', '', 1) for item in lines[ind :]]
        data[0] = re.sub("def.*\(cls, ", "def f_filled (", data[0])
        out_func = ''.join(data)

    try:
        with open('data/evaluation/geeks_for_geeks_successful_test_scripts/python/' + desc) as f:
            lines = f.readlines()
            lines.insert(lines.index('#TOFILL\n') + 1, out_func + '\n')

        with open('experiment/java-python/j2pytests/' + entry.name, 'w') as f:
            f.write(''.join(lines))
    except FileNotFoundError as e:
        print(e)
        continue

In [ ]:
import subprocess
import pandas as pd
import IPython

blocklist = ['for__0__MOVE_ZEROES_END_ARRAY.py', 'if__0__CHECK_IF_A_NUMBER_IS_POWER_OF_ANOTHER_NUMBER.py', 'if__1__MODULUS_TWO_FLOAT_DOUBLE_NUMBERS.py', 'if__0__ALTERNATIVE_SORTING.py', 'if__2__MODULUS_TWO_FLOAT_DOUBLE_NUMBERS.py', 'while__0__COUNT_FACTORIAL_NUMBERS_IN_A_GIVEN_RANGE.py', 'while__1__K_TH_ELEMENT_TWO_SORTED_ARRAYS.py']
d = {'key': [], 'mutator': [], 'number': [], 'computational accuracy': []}

for entry in os.scandir('experiment/java-python/test/tests'):
    if entry.name not in blocklist:
        print(entry.name)
        out = subprocess.run(['python3', entry.path], capture_output=True).stdout
        out = out.decode()
        if out:
            if out[0] == '#':
                unkilled = float(out.split(', ')[0].split(' ')[1])
                total = float(out.split(', ')[1].split('\n')[0])
                
                d['key'].append(entry.name.split('.')[0].split('__')[2])
                d['mutator'].append(entry.name.split('.')[0].split('__')[0])
                d['number'].append(entry.name.split('.')[0].split('__')[1])
                d['computational accuracy'].append(unkilled / total)

        IPython.display.clear_output()

df = pd.DataFrame(data=d)
df

In [ ]:
df3 = pd.concat([df, df2])
df3.to_csv('experiment/java-python/result/individual_mutants_result.csv')